In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_convnext_like_model(input_dim, num_classes, sliding_window_size=3):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    x = reshaped_input
    
    # Simulate ConvNeXt block using Conv1D layers
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Global pooling and classification head
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_convnext_like_model(train_X.shape[1], test_y.shape[1])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [6]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0012
test_accuracy: 0.9996
balanced_accuracy: 0.9912
roc_auc: 1.0000
pr_auc: 1.0000
f2: 0.9996
mcc: 0.9988
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 9117.0000
1_precision: 1.0000
1_recall: 1.0000
1_f1-score: 1.0000
1_support: 10.0000
2_precision: 0.9143
2_recall: 0.9412
2_f1-score: 0.9275
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 1.0000
4_recall: 0.9985
4_f1-score: 0.9992
4_support: 653.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9996
macro avg_precision: 0.9877
macro avg_recall: 0.9912
macro avg_f1-score: 0.9894
macro avg_support: 11600.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.9996
weighted avg_f1-score: 0.9996
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs: